In [1]:
import PyPDF2
import os
import glob
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import numpy as np

def convert_pdf_to_string(file_path):

    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return(output_string.getvalue())


class ProgBar:

    def __init__(self, n_elements,int_str):
        
        import sys

        self.n_elements = n_elements
        self.progress = 0

        print(int_str)

        # initiallizing progress bar

        info = '{:.2f}% - {:d} of {:d}'.format(0,0,n_elements)

        formated_bar = ' '*int(50)

        sys.stdout.write("\r")

        sys.stdout.write('[%s] %s' % (formated_bar,info))

        sys.stdout.flush()

    def update(self,prog_info=None):
        
        import sys

        if prog_info == None:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d}'.format(percent*2,self.progress,self.n_elements)

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


        else:

            self.progress += 1

            percent = (self.progress)/self.n_elements * 100 / 2

            info = '{:.2f}% - {:d} of {:d} '.format(percent*2,self.progress,self.n_elements) + prog_info

            formated_bar = '-'* int (percent) + ' '*int(50-percent)

            sys.stdout.write("\r")

            sys.stdout.write('[%s] %s' % (formated_bar,info))

            sys.stdout.flush()


def add_line(folder,line):

    # Open the file in append & read mode ('a+')

    with open(folder+"/conv.txt", "a+") as file_object:

        # Move read cursor to the start of file.
        file_object.seek(0)

        # If file is not empty then append '\n'
        data = file_object.read(100)

        if len(data) > 0 :
            file_object.write("\n")
            
        # Append text at the end of file
        file_object.write(line)


def extract_figures (folder):

    import fitz
    doc = fitz.open(folder + "/slide.pdf")

    for i,img in enumerate(doc.get_page_images(0)):
        xref = img[0]
        print(img)
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.save(folder + "/figures/%s.png" % (i, xref))
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.save(folder + "/figures/%s.png" % (i, xref))
            pix1 = None
        pix = None



In [10]:
extract_figures('Capitulo_7')

(100, 0, 281, 293, 8, 'Indexed', '', 'Im100', 'FlateDecode')
(107, 0, 614, 297, 8, 'DeviceRGB', '', 'Im107', 'DCTDecode')
(114, 0, 782, 263, 8, 'DeviceRGB', '', 'Im114', 'DCTDecode')
(118, 0, 614, 318, 8, 'DeviceRGB', '', 'Im118', 'DCTDecode')
(122, 0, 611, 231, 8, 'DeviceRGB', '', 'Im122', 'DCTDecode')
(126, 0, 693, 200, 8, 'DeviceRGB', '', 'Im126', 'DCTDecode')
(13, 0, 624, 209, 8, 'DeviceRGB', '', 'Im13', 'DCTDecode')
(139, 0, 605, 180, 8, 'DeviceRGB', '', 'Im139', 'DCTDecode')
(14, 0, 624, 209, 8, 'DeviceRGB', '', 'Im14', 'DCTDecode')
(146, 0, 607, 340, 8, 'DeviceRGB', '', 'Im146', 'DCTDecode')
(159, 0, 567, 444, 8, 'DeviceRGB', '', 'Im159', 'DCTDecode')
(18, 0, 321, 164, 8, 'DeviceRGB', '', 'Im18', 'DCTDecode')
(184, 0, 643, 350, 8, 'DeviceRGB', '', 'Im184', 'DCTDecode')
(197, 0, 472, 402, 8, 'DeviceRGB', '', 'Im197', 'DCTDecode')
(201, 0, 472, 404, 8, 'DeviceRGB', '', 'Im201', 'DCTDecode')
(202, 0, 210, 149, 8, 'DeviceRGB', '', 'Im202', 'DCTDecode')
(22, 0, 321, 164, 8, 'DeviceRG

In [40]:
import fitz

doc = fitz.open("Capitulo_7/slide.pdf")
blocks = doc[2].get_text('blocks')

for bck in blocks:
    bck_type = bck[4].split(':')[0]
    if bck_type == '<image':
        print(int(bck[4].split(',')[-1].split(': ')[-1].split('>')[0]))

8
8
8
8


In [ ]:

for i,img in enumerate(doc.get_page_images(0)):
    xref = img[0]
    print(img)
    pix = fitz.Pixmap(doc, xref)
    if pix.n < 5:       # this is GRAY or RGB
        pix.save(folder + "/figures/p%s-%s.png" % (i, xref))
    else:               # CMYK: convert to RGB first
        pix1 = fitz.Pixmap(fitz.csRGB, pix)
        pix1.save(folder + "/figures/p%s-%s.png" % (i, xref))
        pix1 = None
    pix = None


In [32]:
pages_struct = {}

folder = 'Capitulo_7'

reader = PyPDF2.PdfFileReader(folder + '/slide.pdf')

pdf_n_pages = reader.numPages

writer = PyPDF2.PdfFileWriter()

### Reading text and saving in the dictionary

# Initializing progress bar

bar = ProgBar(pdf_n_pages,'Reading text and saving in the dictionary...')


for pg in range(pdf_n_pages):

    pages_struct[pg] = {
        'figures': [],
        'text': 0,
        'equations': []
    }

    writer = PyPDF2.PdfFileWriter()

    output_filename = 'transition.pdf'

    page = reader.getPage(pg)

    writer.addPage(page)

    with open(output_filename, 'wb') as output:
        writer.write(output)

    del writer

    text = convert_pdf_to_string('transition.pdf')

    text = text.replace('\x0c',' ')
    text = text.replace('%','\%')
    text = text.split('\n')
    text = text[5:]

    pages_struct[pg]['text'] = text

    # getting figures

    figures_list = save_figures(folder,page,pg)

    bar.update()


# saving formated equations on the dictionary

# listing equation's figures

figures_list  = glob.glob(folder+'/equations/*')

# sorting list o files

page_list = []

# getting list of pages with equations

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    
    pg,_= name.split('-')

    # appending pages with eaqutions

    page_list.append(int(pg))

# sorting list of pages 

page_list = list(np.sort(page_list))

# creating dictionary for appending the list  of figures in each page

eq_dict = {}

for pg in page_list:

    eq_dict[pg] = []


# getting list of eqautions in each page

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')
    pg,num = name.split('-')

    # appending numbe of the equations in this page

    eq_dict[int(pg)].append(int(num))

    # sorting equations

    eq_dict[int(pg)] = list(np.sort(eq_dict[int(pg)]))

# setting count of equations

k = 1

for pg in eq_dict:

    for  num in eq_dict[pg]:
        
        # extracting the page and number from dictionary

        name = '-'.join((str(pg),str(num)))

        # writing LaTex syntax

        figure_call = [
            '\\begin{figure}[h!]',
            '\centering',
            ('\includegraphics[scale=0.5]{Pictures/' + folder + '/equations/' + name + '.png}'),
            ('\caption{Equação' + str(k) + '}'),
            ('\label{fig:eq' + str(k) + '}'),
            '\end{figure}',
            ' ',
            ' '
        ]

        # incremeanting count

        k += 1

        # appending formated text in the dicctionary

        for i in range(len(figure_call)): 
            pages_struct[int(pg)-1]['equations'].append(figure_call[i])



"""
# Reading figures and saving in the dictionary

figures_list  = glob.glob(folder+'/figures/*')

for i in range(len(figures_list)): 

    # extracting the page and format of the file's names

    name,form = figures_list[i].split('/')[-1].split('.')

    try:
    
        pg,_= name.split('-')

    except:

        pg = name

    figure_call = [
        '\\begin{figure}[h!]',
        '\centering',
        ('\includegraphics[scale=0.5]{Pictures/' + folder + '/figures/' + name + '.' + form + '}'),
        '\caption{Caption}',
        '\label{fig:my_label}',
        '\end{figure}'
        ''
        ''
    ]

    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])

    """

Reading text and saving in the dictionary...
[                                                  ] 0.00% - 0 of 70{'/Type': '/XObject', '/Subtype': '/Image', '/Width': 281, '/Height': 293, '/BitsPerComponent': 8, '/Filter': '/FlateDecode', '/ColorSpace': ['/Indexed', '/DeviceRGB', 255, b'\x00\x00\x00\x00\x00U\x00\x00\xaa\x00\x00\xff\x00$\x00\x00$U\x00$\xaa\x00$\xff\x00I\x00\x00IU\x00I\xaa\x00I\xff\x00m\x00\x00mU\x00m\xaa\x00m\xff\x00\x92\x00\x00\x92U\x00\x92\xaa\x00\x92\xff\x00\xb6\x00\x00\xb6U\x00\xb6\xaa\x00\xb6\xff\x00\xdb\x00\x00\xdbU\x00\xdb\xaa\x00\xdb\xff\x00\xff\x00\x00\xffU\x00\xff\xaa\x00\xff\xff$\x00\x00$\x00U$\x00\xaa$\x00\xff$$\x00$$U$$\xaa$$\xff$I\x00$IU$I\xaa$I\xff$m\x00$mU$m\xaa$m\xff$\x92\x00$\x92U$\x92\xaa$\x92\xff$\xb6\x00$\xb6U$\xb6\xaa$\xb6\xff$\xdb\x00$\xdbU$\xdb\xaa$\xdb\xff$\xff\x00$\xffU$\xff\xaa$\xff\xffI\x00\x00I\x00UI\x00\xaaI\x00\xffI$\x00I$UI$\xaaI$\xffII\x00IIUII\xaaII\xffIm\x00ImUIm\xaaIm\xffI\x92\x00I\x92UI\x92\xaaI\x92\xffI\xb6\x00I\xb6UI\xb6\xaaI\xb6\x

"\n# Reading figures and saving in the dictionary\n\nfigures_list  = glob.glob(folder+'/figures/*')\n\nfor i in range(len(figures_list)): \n\n    # extracting the page and format of the file's names\n\n    name,form = figures_list[i].split('/')[-1].split('.')\n\n    try:\n    \n        pg,_= name.split('-')\n\n    except:\n\n        pg = name\n\n    figure_call = [\n        '\\begin{figure}[h!]',\n        '\\centering',\n        ('\\includegraphics[scale=0.5]{Pictures/' + folder + '/figures/' + name + '.' + form + '}'),\n        '\\caption{Caption}',\n        '\\label{fig:my_label}',\n        '\\end{figure}'\n        ''\n        ''\n    ]\n\n    for i in range(len(figure_call)): pages_struct[int(pg)-1]['figures'].append(figure_call[i])\n\n    "

In [18]:
import fitz
doc = fitz.open("Capitulo_7/slide.pdf")
doc

Document('Capitulo_7/slide.pdf')

In [19]:
len(doc)

70

In [25]:
for i in range(len(doc)):
    page = doc[i]
    figures_list = page.get_image_bbox()
    print(len(figures_list))


TypeError: get_image_bbox() missing 1 required positional argument: 'name'

In [26]:
for page in doc:
    pix = page.getPixmap()
    pix.writeImage("page-%i.png" % page.number)

FitzDeprecation: 'getPixmap' removed from class 'Page' after v1.19.0 - use 'get_pixmap'. [__init__.py:169]
FitzDeprecation: 'writeImage' removed from class 'Pixmap' after v1.19.0 - use 'save'. [__init__.py:169]


In [ ]:

for i in range(len(doc)):

    for j,img in enumerate(doc.get_page_images(i)):
    
        file_name = folder + '/figures/' + str(page_num) + '_' + str(j) + ".png"

        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.save(file_name)
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.save(file_name)
            pix1 = None
        pix = None

In [5]:
### Saving results in a txt file 

for pg in pages_struct:
    
    # appending text

    for txt in pages_struct[pg]['text']:
        add_line(folder,txt)

    # appending equations

    for eq in pages_struct[pg]['equations']:
        add_line(folder,eq)

    # appending figures

    for fig in pages_struct[pg]['figures']:
        add_line(folder,fig)

    # adding new page

    add_line(folder,'\\newpage')